In [18]:
import pandas as pd


df = pd.read_csv('data/georgo_ho_clues.csv')

df  = df.loc[df['source'] == 
'times_xwd_times']


In [19]:
len(df)

101240

In [29]:

for i, row in df.iterrows():
    clue = row['clue'].split('(')[0].strip()
    definition = str(row['definition'])
    
    df.at[i,'definition_length'] = len(definition.split(' '))
    if clue.startswith(definition.strip()):
        df.at[i,'definition_position'] = 'start'
    elif clue.endswith(definition.strip()):
        df.at[i,'definition_position'] = 'end'
    else:
        df.at[i,'definition_position'] = 'middle'

    df.at[i,'def_length_pos_combined'] = str (df.at[i,'definition_length']) + str(df.at[i,'definition_position'])
        

In [30]:
df = df [['clue', 'definition', 'definition_length', 'definition_position', 'answer','def_length_pos_combined']]

In [31]:
df

,clue,definition,definition_length,definition_position,answer,def_length_pos_combined
0,"Acquisitive chap, as we see it (8)",Acquisitive,1.0,start,COVETOUS,1.0start
1,Back yard fencing weak and sagging (6),sagging,1.0,end,DROOPY,1.0end
2,"Stripping off uniform, love holding colonel's ...",Stripping,1.0,start,UNCLOTHING,1.0start
3,Without a mark where they should be gained (4),where they should be gained,5.0,end,EXAM,5.0end
4,"Put a stop to Rugby's foul school leader (5,2,...",Put a stop to,4.0,start,KNOCK ON THE HEAD,4.0start
...,...,...,...,...,...,...
627156,Copper abandoned remote obsession (8),Copper abandoned,2.0,start,FARTHING,2.0start
627157,Stretch of land without a green? Keep out of i...,Keep out of it,4.0,end,NO-GO AREA,4.0end
627158,"Kept short socks, initially coloured (7)",Kept short,2.0,start,STINTED,2.0start
627159,In surprise attack start to beat into a pulp (6),surprise attack,2.0,middle,AMBUSH,2.0middle


In [28]:
df['definition_length'].value_counts()

definition_length
1.0     49004
2.0     26171
3.0     13294
4.0      6720
5.0      2936
6.0      1487
7.0       793
8.0       397
9.0       248
10.0      114
11.0       51
12.0       20
13.0        4
14.0        1
Name: count, dtype: int64

In [27]:
df['definition_position'].value_counts()

definition_position
start     48176
end       40154
middle    12910
Name: count, dtype: int64

In [32]:
len(df['def_length_pos_combined'].value_counts())

37

In [42]:
import math
sampled_df = []
for k,v in df['def_length_pos_combined'].value_counts().items():
    n_elements = math.ceil(v/len(df) * 1000)
    n_elements = int(n_elements)
    sampled_df.append(df.loc[df['def_length_pos_combined'] == k].sample(n_elements))


sampled_df = pd.concat(sampled_df)

sampled_df = sampled_df.sample(1000).reset_index(drop=True)

In [48]:
sampled_df = sampled_df.rename(columns={'clue':'Clue', 'definition':'Definition', 'answer':'Answer'})
sampled_df.to_csv('data/georgo_ho_clues_sampled.csv', index=False)

In [49]:
sampled_df['definition_position'].value_counts(), sampled_df['definition_length'].value_counts()

(definition_position
 start     470
 end       397
 middle    133
 Name: count, dtype: int64,
 definition_length
 1.0     473
 2.0     255
 3.0     131
 4.0      68
 5.0      30
 6.0      15
 7.0      10
 8.0       5
 9.0       4
 10.0      2
 12.0      2
 13.0      2
 11.0      2
 14.0      1
 Name: count, dtype: int64)

In [17]:
import pandas as pd
results = []

with open('results/definition_wordplay_extraction/chatgpt_eval_def_1k_samples.txt', 'r') as f:
    for line in f:
        results.append(line)

sampled_df = pd.read_csv('data/georgo_ho_clues_sampled.csv')
results = results[25:1025]


In [24]:
sampled_df['definition_position'].value_counts()

definition_position
start     470
end       397
middle    133
Name: count, dtype: int64

In [25]:


cnt = 0 
begin = 0
end = 0
mid = 0
for i,res in enumerate(results):
    if ('✅') in res:
        cnt += 1
        assert res.split('|')[0].split('Clue:')[1].strip() == sampled_df.at[i,'Clue']
        if sampled_df.at[i,'definition_position'] == 'start':
            begin += 1
        elif sampled_df.at[i,'definition_position'] == 'end':
            end += 1
        else:
            mid += 1

print(cnt)
print(begin/294)
print(end/294)
print(mid/294)

294
0.6904761904761905
0.20068027210884354
0.10884353741496598
